# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [41]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

from patsy import dmatrices
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

%matplotlib notebook

In [27]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [10]:
df.describe()

,Unnamed: 0,id_cliente,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,renda
count,15000.000000,15000.000000,15000.000000,15000.000000,12427.000000,15000.000000,15000.000000
mean,7499.500000,8304.871400,0.432333,43.882333,7.722635,2.206400,5697.287057
std,4330.271354,4797.780446,0.746631,11.273155,6.711189,0.909792,8266.816289
min,0.000000,1.000000,0.000000,22.000000,0.117808,1.000000,118.710000
25%,3749.750000,4181.000000,0.000000,34.000000,2.973973,2.000000,2026.110000
50%,7499.500000,8297.000000,0.000000,43.000000,6.013699,2.000000,3499.720000
75%,11249.250000,12403.000000,1.000000,53.000000,10.120548,3.000000,6392.167500
max,14999.000000,16649.000000,14.000000,68.000000,42.906849,15.000000,245141.670000


In [11]:
df.dtypes

Unnamed: 0                 int64
data_ref                  object
id_cliente                 int64
sexo                      object
posse_de_veiculo            bool
posse_de_imovel             bool
qtd_filhos                 int64
tipo_renda                object
educacao                  object
estado_civil              object
tipo_residencia           object
idade                      int64
tempo_emprego            float64
qt_pessoas_residencia    float64
renda                    float64
dtype: object

In [28]:
df = df.drop(['data_ref', 'Unnamed: 0'], axis=1)

# Convertendo variáveis categóricas em dummies
categorical_vars = ['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'tipo_renda',
                    'educacao', 'estado_civil', 'tipo_residencia']
df = pd.get_dummies(df, columns=categorical_vars)

# Tratando valores não-finitos
df['tempo_emprego'] = df['tempo_emprego'].fillna(0)
df['qt_pessoas_residencia'] = df['qt_pessoas_residencia'].fillna(0)

# Convertendo as variáveis em int64
int_vars = ['tempo_emprego', 'qt_pessoas_residencia']
df[int_vars] = df[int_vars].astype('int64')

In [29]:
# 1)
X = df.drop('renda', axis=1)
y = df['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [30]:
# 2)
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_scores = []

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    
    y_pred = ridge.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)
    
    print(f"Alpha: {alpha}, R2: {r2}")

best_model_index = np.argmax(r2_scores)
best_alpha = alphas[best_model_index]

print(f"\nMelhor modelo: Alpha={best_alpha}, R2={r2_scores[best_model_index]}")

Alpha: 0, R2: 0.26671477964697776
Alpha: 0.001, R2: 0.2667154549842734
Alpha: 0.005, R2: 0.26671548394424505
Alpha: 0.01, R2: 0.266715520108435
Alpha: 0.05, R2: 0.2667158080074843
Alpha: 0.1, R2: 0.2667161644235615

Melhor modelo: Alpha=0.1, R2=0.2667161644235615


In [35]:
# 3)
alphas = [0.001, 0.005, 0.01, 0.05, 0.1]
r2_scores = []


for alpha in alphas:
    
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    
    y_pred = lasso.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)
    
    print(f"Alpha: {alpha}, R2: {r2}")

best_model_index = np.argmax(r2_scores)
best_alpha = alphas[best_model_index]

print(f"\nMelhor modelo: Alpha={best_alpha}, R2={r2_scores[best_model_index]}")

/Users/rodrigomoraes/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.830e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


Alpha: 0.001, R2: 0.2667155454245902


/Users/rodrigomoraes/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.705e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


Alpha: 0.005, R2: 0.26671603258949617


/Users/rodrigomoraes/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.556e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


Alpha: 0.01, R2: 0.2667165497314753


/Users/rodrigomoraes/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.796e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


Alpha: 0.05, R2: 0.26672129795864274
Alpha: 0.1, R2: 0.26672748038120553

Melhor modelo: Alpha=0.1, R2=0.26672748038120553


/Users/rodrigomoraes/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.139e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


In [36]:
# 4)

# Função para realizar o modelo stepwise
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out=0.05, 
                       verbose=True):
    included = list(initial_list)
    while True:
        changed = False
        
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=float)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Adicionando variável "{best_feature}" com p-value {best_pval:.4f}')
        
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            changed = True
            if verbose:
                print(f'Removendo variável "{worst_feature}" com p-value {worst_pval:.4f}')
        
        if not changed:
            break
    
    return included

selected_features = stepwise_selection(X_train, y_train)

model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()

y_pred = model.predict(sm.add_constant(X_test[selected_features]))

r2 = r2_score(y_test, y_pred)

print(f"Melhor modelo: R2 = {r2}")

Adicionando variável "tempo_emprego" com p-value 0.0000
Adicionando variável "sexo_M" com p-value 0.0000
Adicionando variável "sexo_F" com p-value 0.0000
Adicionando variável "tipo_renda_Pensionista" com p-value 0.0000
Adicionando variável "tipo_renda_Empresário" com p-value 0.0000
Adicionando variável "idade" com p-value 0.0000
Adicionando variável "educacao_Superior completo" com p-value 0.0000
Melhor modelo: R2 = 0.2660642527876218


#### 5) Conclusão da comparação dos parâmetros
Considerando o $R^2$ como métrica de avaliação, o melhor desempenho é alcançado pela regressão Lasso com alpha=0.1, apresentando um $R^2$ de 0.26672748038120553.

In [37]:
# 6)

# Selecionar as variáveis numéricas do DataFrame
numeric_vars = df.select_dtypes(include=np.number).columns.tolist()

# Copiar o DataFrame original para preservar os dados originais
df_transformed = df.copy()

# Aplicar a transformação logarítmica nas variáveis numéricas
df_transformed[numeric_vars] = np.log1p(df[numeric_vars])

# Verificar o resultado
df_transformed.head()

,id_cliente,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,renda,sexo_F,sexo_M,posse_de_veiculo_False,posse_de_veiculo_True,...,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Aluguel,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,9.619598,0.000000,3.295837,1.945910,0.693147,8.994835,0.693359,0.000000,0.693359,0.000000,...,0.0,0.693359,0.0,0.0,0.0,0.693359,0.0,0.0,0.0,0.000000
1,9.207236,0.000000,3.367296,2.079442,1.098612,7.524642,0.000000,0.693359,0.000000,0.693359,...,0.0,0.000000,0.0,0.0,0.0,0.693359,0.0,0.0,0.0,0.000000
2,8.369389,0.000000,3.583519,0.000000,1.098612,7.720856,0.693359,0.000000,0.000000,0.693359,...,0.0,0.000000,0.0,0.0,0.0,0.693359,0.0,0.0,0.0,0.000000
3,9.272376,0.693147,3.433987,1.609438,1.386294,8.795093,0.693359,0.000000,0.693359,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.693359,0.0,0.0,0.0,0.000000
4,8.862908,0.000000,3.526361,1.609438,0.693147,8.776008,0.000000,0.693359,0.000000,0.693359,...,0.0,0.693359,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.693359


In [39]:
# 6)
X = df.drop('renda', axis=1)
y = df['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [40]:
# 6)
alphas = [0.001, 0.005, 0.01, 0.05, 0.1]
r2_scores = []

# Loop pelos diferentes valores de alpha
for alpha in alphas:
    # Instanciar o modelo Lasso com o valor de alpha
    lasso = Lasso(alpha=alpha)
    
    # Treinar o modelo usando os dados de treinamento
    lasso.fit(X_train, y_train)
    
    # Fazer previsões na base de testes
    y_pred = lasso.predict(X_test)
    
    # Calcular o coeficiente de determinação (R2)
    r2 = r2_score(y_test, y_pred)
    
    # Adicionar o valor de R2 à lista
    r2_scores.append(r2)
    
    # Imprimir o resultado
    print(f"Alpha: {alpha}, R2: {r2}")

# Encontrar o índice do valor máximo de R2
best_model_index = np.argmax(r2_scores)

# Obter o valor de alpha correspondente ao melhor modelo
best_alpha = alphas[best_model_index]

print(f"\nMelhor modelo: Alpha={best_alpha}, R2={r2_scores[best_model_index]}")

/Users/rodrigomoraes/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.830e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(
/Users/rodrigomoraes/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.705e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


Alpha: 0.001, R2: 0.2667155454245902
Alpha: 0.005, R2: 0.26671603258949617


/Users/rodrigomoraes/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.556e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(
/Users/rodrigomoraes/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.796e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


Alpha: 0.01, R2: 0.2667165497314753
Alpha: 0.05, R2: 0.26672129795864274
Alpha: 0.1, R2: 0.26672748038120553

Melhor modelo: Alpha=0.1, R2=0.26672748038120553


/Users/rodrigomoraes/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.139e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


In [42]:
# 7)
# Instanciar o modelo de árvore de regressão
tree = DecisionTreeRegressor()

# Treinar o modelo usando os dados de treinamento
tree.fit(X_train, y_train)

# Fazer previsões na base de testes
y_pred = tree.predict(X_test)

# Calcular o coeficiente de determinação (R2)
r2 = r2_score(y_test, y_pred)

# Imprimir o resultado
print(f"R2 da árvore de regressão: {r2}")

R2 da árvore de regressão: 0.224470702449466
